# SQL Анализ для создания приложения-читалки для книг.

## 1 Цели исследования

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

In [1]:
!pip install psycopg2-binary

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

In [4]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## 2 Исследуем таблицы

### 2.1 **Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [5]:
query = ''' SELECT *
            FROM books 
            LIMIT 5 '''
books = pd.io.sql.read_sql(query, con = engine)
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### 2.2 **Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [6]:
query = ''' SELECT *
            FROM authors 
            LIMIT 5 '''
authors = pd.io.sql.read_sql(query, con = engine)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### 2.3 **Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

In [7]:
query = ''' SELECT *
            FROM publishers 
            LIMIT 5 '''
publishers = pd.io.sql.read_sql(query, con = engine)
publishers



,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### 2.4 **Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [8]:
query = ''' SELECT *
            FROM ratings 
            LIMIT 5 '''
ratings = pd.io.sql.read_sql(query, con = engine)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### 2.5 **Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [9]:
query = ''' SELECT *
            FROM reviews 
            LIMIT 5 '''
reviews = pd.io.sql.read_sql(query, con = engine)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## 3 Решение задач

### 3.1 Посчитайте, сколько книг вышло после 1 января 2000 года;

In [10]:
query = ''' SELECT COUNT(book_id)
            FROM books
            WHERE publication_date > '2000-01-01'; '''

pd.io.sql.read_sql(query, con = engine)

,count
0,819


**Вывод:**   819  книг вышло после 1 января 2000 года

### 3.2 Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [15]:
query = ''' SELECT b.title,
                   COUNT(DISTINCT r.review_id),
                   ROUND(AVG(rt.rating), 2) AS avg_rat
            FROM reviews AS r
            RIGHT OUTER JOIN books AS b ON r.book_id = b.book_id
            LEFT OUTER JOIN ratings AS rt ON b.book_id = rt.book_id
            GROUP BY b.book_id
            ORDER BY count DESC; '''

pd.io.sql.read_sql(query, con = engine)

,title,count,avg_rat
0,Twilight (Twilight #1),7,3.66
1,Water for Elephants,6,3.98
2,The Glass Castle,6,4.21
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,The Natural Way to Draw,0,3.00
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,Essential Tales and Poems,0,4.00


**Вывод:** По среднему рейтингу в пятерку лучших вошли :
- Arrows of the Queen (Heralds of Valdemar #1)		5.00
- The Walking Dead Book One (The Walking Dead #		5.00
- Light in August	5.00
- Wherever You Go There You Are: Mindfulness Me		5.00
- Captivating: Unveiling the Mystery of a Woman'	5.00

### 3.3 Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры;

In [12]:
query = ''' SELECT p.publisher,
                   COUNT(b.book_id)
            FROM publishers AS p
            LEFT JOIN books AS b ON p.publisher_id = b.publisher_id
            WHERE b.book_id IN (SELECT book_id
                                FROM books
                                WHERE num_pages >= 50)
            GROUP BY p.publisher_id
            ORDER BY COUNT(b.book_id) DESC
            LIMIT 1; '''

pd.io.sql.read_sql(query, con = engine)

,publisher,count
0,Penguin Books,42


**Вывод:** Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц.

### 3.4 Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [13]:
query = ''' SELECT a.author,
                   ROUND(AVG(rt.rating), 2)
            FROM authors AS a
            LEFT JOIN books AS b ON a.author_id = b.author_id
            LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
            WHERE b.book_id IN (SELECT book_id
                                FROM ratings
                                GROUP BY book_id
                                HAVING COUNT(rating) >= 50)
            GROUP BY a.author
            ORDER BY AVG(rt.rating) DESC
            LIMIT 1; '''

pd.io.sql.read_sql(query, con = engine)

,author,round
0,J.K. Rowling/Mary GrandPré,4.29


**Вывод:** Aвтор с самой высокой средней оценкой книг, которые оценены более 50 раз.
`J.K. Rowling/Mary GrandPré	4.29`

### 3.5 Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [14]:
query = ''' WITH t1 AS (
SELECT username,
       COUNT(rating) AS cnt_rat
FROM ratings
GROUP BY username
HAVING COUNT(rating) > 50),

t2 AS (
SELECT username,
       COUNT(review_id) AS cnt_rev
FROM reviews
GROUP BY 1)

SELECT AVG(cnt_rev)
FROM t1
LEFT JOIN t2 ON t1.username = t2.username '''

pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


**Вывод:**  = 24 обзора в среднем, от пользователей, которые поставили больше 50 оценок.

<div style="border-radius: 15px; box-shadow: 2px 2px 4px; border: 1px solid; background:#bdc3c7; padding: 20px"> <h7 style="color:black; margin-bottom:20px"> 
    ОБЩИЙ вывод: 
    
    - 819 книг вышло после 1 января 2000 года 
    
    - По среднему рейтингу в пятерку лучших вошли :
    - Arrows of the Queen (Heralds of Valdemar #1) 5.00
    - The Walking Dead Book One (The Walking Dead # 5.00
    - Light in August 5.00
    - Wherever You Go There You Are: Mindfulness Me 5.00
    - Captivating: Unveiling the Mystery of a Woman' 5.00 
    
    - Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц
    
    - Aвтор с самой высокой средней оценкой книг, которые оценены более 50 раз. J.K. Rowling/Mary GrandPré    4.29
    
    - = 24 обзора в среднем, от пользователей, которые поставили больше 50 оценок.